In [1]:
#%%
from LiouvilleLanczos.Quantum_computer.QC_lanczos import Liouvillian_slo,inner_product_slo,sum_slo
from LiouvilleLanczos.Quantum_computer.Hamiltonian import Line_Hubbard,BoundaryCondition
from LiouvilleLanczos.Lanczos import Lanczos
from LiouvilleLanczos.matrix_impl import MatrixState_inner_product,Matrix_Liouvillian,Matrix_sum
from LiouvilleLanczos.Quantum_computer.Mapping import find_best_layout
from LiouvilleLanczos.Green import CF_Green
from qiskit.primitives import StatevectorEstimator as pEstimator

from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.operators import FermionicOp
from qiskit.quantum_info.operators import SparsePauliOp
from qiskit import QuantumCircuit
import numpy as np
from qiskit import transpile
from datetime import datetime
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

In [4]:
U = 4
mapper = JordanWignerMapper()
Ham = Line_Hubbard(-1,U/2,U,2,boundary_condition=BoundaryCondition.OPEN)
#up spin site 1
C2 = FermionicOp(
    {
        "+_2": 1,
    },
    num_spin_orbitals=4,
)
#up spin site 0
C0 = FermionicOp(
    {
        "+_0": 1,
    },
    num_spin_orbitals=4,
)
C0_mat = mapper.map(C0).to_matrix()
C2_mat = mapper.map(C2).to_matrix()
C0101 = mapper.map(FermionicOp(
    {
        "+_0 +_1": 1,
    },
    num_spin_orbitals=4,
)).to_matrix()
C0110 = mapper.map(FermionicOp(
    {
        "+_0 +_3": 1,
    },
    num_spin_orbitals=4,
)).to_matrix()
C1001 = mapper.map(FermionicOp(
    {
        "+_1 +_2": 1,
    },
    num_spin_orbitals=4,
)).to_matrix()
C1010 = mapper.map(FermionicOp(
    {
        "+_2 +_3": 1,
    },
    num_spin_orbitals=4,
)).to_matrix()
print(Ham)

Fermionic Operator
number spin orbitals=4, number terms=10
  -1 * ( +_0 -_2 )
+ 1 * ( -_0 +_2 )
+ -2.0 * ( +_0 -_0 )
+ -2.0 * ( +_2 -_2 )
+ -1 * ( +_1 -_3 )
+ 1 * ( -_1 +_3 )
+ -2.0 * ( +_1 -_1 )
+ -2.0 * ( +_3 -_3 )
+ 4 * ( +_0 -_0 +_1 -_1 )
+ 4 * ( +_2 -_2 +_3 -_3 )


In [5]:
#%% Ground state circuit, obtained by inspection of analytical wavefunction
bt = 0.7854074074074073
GS_analytical = QuantumCircuit(4)
GS_analytical.h(0)
GS_analytical.x(1)
GS_analytical.cx(0,1)
GS_analytical.ry(bt,2)
GS_analytical.x(3)
GS_analytical.cx(2,3)
GS_analytical.cx(1,3)
GS_analytical.cx(1,2)
GS_analytical.cz(1,2)
GS_analytical.swap(1,2)
GS_analytical.draw('mpl')

#%% Sanity check: compare matrix ground energy with simulated estimator ground energy.
Hmat = mapper.map(Ham).to_matrix()
estimator = pEstimator()
qubit_converter = (JordanWignerMapper())
HHam = qubit_converter.map(Ham)
E,S = np.linalg.eigh(Hmat)
GS_mat = S[:,0]
print(E[0])
print(estimator.run([(GS_analytical,HHam)]).result()[0].data.evs)

-4.82842712474619
-4.8284271246253425
